In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import numpy as np

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/12th Grade/Machine Learning/Data/Reservations/train.csv')

In [ ]:
df.head()

,id,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,0,2,0,0,2,1,0,0,9,2018,1,14,1,1,11,0,67.50,0,0
1,1,2,0,1,2,0,0,0,117,2018,7,29,0,0,0,0,72.25,0,0
2,2,2,0,0,1,0,0,0,315,2018,12,2,0,0,0,0,52.00,0,0
3,3,1,0,0,2,1,0,0,32,2018,12,1,1,0,0,0,56.00,0,0
4,4,2,0,1,0,0,0,0,258,2018,10,16,0,0,0,0,100.00,0,1


In [ ]:
df.loc[df['avg_price_per_room']==0]

,id,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
192,192,1,0,0,2,2,0,0,1,2017,12,10,1,0,0,0,0.0,1,0
234,234,2,0,0,1,0,0,2,0,2018,4,15,1,0,0,0,0.0,0,0
362,362,1,0,0,1,0,0,2,37,2017,12,2,4,0,0,0,0.0,1,0
527,527,2,0,0,0,0,0,0,75,2018,11,17,1,0,0,0,0.0,3,0
617,617,1,0,1,2,2,0,0,80,2018,9,26,1,0,0,0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41802,41802,1,0,0,0,1,0,0,36,2018,10,27,1,0,0,0,0.0,0,1
41812,41812,2,0,0,2,2,0,0,11,2017,10,10,4,0,0,0,0.0,0,0
41820,41820,1,0,0,0,2,0,0,6,2018,4,10,1,0,0,0,0.0,0,1
41965,41965,1,0,0,1,0,0,0,4,2018,9,10,4,1,0,1,0.0,1,0


In [ ]:
df['arrival_year'].unique()

array([2018, 2017])

In [ ]:
df.dtypes

id                                        int64
no_of_adults                              int64
no_of_children                            int64
no_of_weekend_nights                      int64
no_of_week_nights                         int64
type_of_meal_plan                         int64
required_car_parking_space                int64
room_type_reserved                        int64
lead_time                                 int64
arrival_year                              int64
arrival_month                             int64
arrival_date                              int64
market_segment_type                       int64
repeated_guest                            int64
no_of_previous_cancellations              int64
no_of_previous_bookings_not_canceled      int64
avg_price_per_room                      float64
no_of_special_requests                    int64
booking_status                            int64
dtype: object

In [ ]:
df.shape

(42100, 19)

In [ ]:
((df['no_of_special_requests']==0) & (df['booking_status']==1)).sum()/42100

0.2781472684085511

In [ ]:
df.columns

Index(['id', 'no_of_adults', 'no_of_children', 'no_of_weekend_nights',
       'no_of_week_nights', 'type_of_meal_plan', 'required_car_parking_space',
       'room_type_reserved', 'lead_time', 'arrival_year', 'arrival_month',
       'arrival_date', 'market_segment_type', 'repeated_guest',
       'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
       'avg_price_per_room', 'no_of_special_requests', 'booking_status'],
      dtype='object')

In [ ]:
df2 = df.copy()

In [ ]:
df2['net_bookings'] = df2['no_of_previous_bookings_not_canceled'] - df2['no_of_previous_cancellations']

In [ ]:
df2['weekend_to_week'] = df2['no_of_weekend_nights'] - df2['no_of_week_nights']

In [ ]:
df2['mo_day'] = df2['arrival_month']*100+df2['arrival_date']

In [ ]:
df2['family_size'] = df2['no_of_adults'] + df2['no_of_children']

In [ ]:
def make_season(row):
    if row['arrival_month']>=3 and row['arrival_month']<6:
      return 1
    elif row['arrival_month']>=6 and row['arrival_month']<9:
      return 2
    elif row['arrival_month']>=9 and row['arrival_month']<12:
      return 3
    else:
      return 0
df2['season'] = df2.apply(make_season, axis=1)

In [ ]:
def make_day(row):
  day = row['arrival_year']-2000
  day += int(day/4)
  day += row['arrival_date']
  code = 0
  if row['arrival_month'] == 2:
    code = 4
  elif row['arrival_month'] == 3:
    code=4
  elif row['arrival_month'] == 4:
    code=0
  elif row['arrival_month'] == 5:
    code=2
  elif row['arrival_month'] == 6:
    code=5
  elif row['arrival_month'] == 7:
    code=0
  elif row['arrival_month'] == 8:
    code=3
  elif row['arrival_month'] == 9:
    code=6
  elif row['arrival_month'] == 10:
    code=1
  elif row['arrival_month'] == 11:
    code=4
  elif row['arrival_month'] == 12:
    code=6
  day+=code
  day = day%7
  return day
df2['day_of_week'] = df2.apply(make_day, axis=1)

In [ ]:
day = 18
day += int(day/4)
day = day%7
print(day)

1


In [ ]:
df2.head()

,id,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,...,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status,net_bookings,weekend_to_week,mo_day,family_size,season,day_of_week
0,0,2,0,0,2,1,0,0,9,2018,...,0,67.50,0,0,-11,-2,114,2,0,1.0
1,1,2,0,1,2,0,0,0,117,2018,...,0,72.25,0,0,0,-1,729,2,2,2.0
2,2,2,0,0,1,0,0,0,315,2018,...,0,52.00,0,0,0,-1,1202,2,0,2.0
3,3,1,0,0,2,1,0,0,32,2018,...,0,56.00,0,0,0,-2,1201,1,0,1.0
4,4,2,0,1,0,0,0,0,258,2018,...,0,100.00,0,1,0,1,1016,2,3,4.0


In [ ]:
df2['day_of_week'].unique()

array([1., 2., 4., 3., 6., 5., 0.])

In [ ]:
features = ['no_of_adults', 'no_of_children', 'family_size', 'no_of_weekend_nights',	'no_of_week_nights', 'weekend_to_week', 'type_of_meal_plan', 'required_car_parking_space',
       'room_type_reserved', 'lead_time', 'mo_day', 'market_segment_type', 'repeated_guest', 'net_bookings', 'no_of_previous_bookings_not_canceled', 'no_of_previous_cancellations',
       'avg_price_per_room', 'day_of_week', 'season', 'no_of_special_requests']

In [ ]:
X = df2[features]
y = df2['booking_status']

In [ ]:
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rf_model = RandomForestClassifier(n_estimators=100, random_state = 42)
rf_model.fit(X_train, y_train)

y_pred2 = rf_model.predict(X_test)
print(accuracy_score(y_test, y_pred2))

0.8049881235154395


In [ ]:
df_new = pd.read_csv('/content/drive/MyDrive/12th Grade/Machine Learning/Data/Reservations/test.csv')

In [ ]:
df_new['net_bookings'] = df_new['no_of_previous_bookings_not_canceled'] - df_new['no_of_previous_cancellations']

In [ ]:
df_new['weekend_to_week'] = df_new['no_of_weekend_nights'] - df_new['no_of_week_nights']

In [ ]:
df_new['mo_day'] = df_new['arrival_month']*100+df_new['arrival_date']

In [ ]:
df_new['family_size'] = df_new['no_of_adults'] + df_new['no_of_children']

In [ ]:
def make_season(row):
    if row['arrival_month']>=3 and row['arrival_month']<6:
      return 1
    elif row['arrival_month']>=6 and row['arrival_month']<9:
      return 2
    elif row['arrival_month']>=9 and row['arrival_month']<12:
      return 3
    else:
      return 0
df_new['season'] = df_new.apply(make_season, axis=1)

In [ ]:
df_new['day_of_week'] = df_new.apply(make_day, axis=1)

In [ ]:
X_new = df_new[features]

In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X, y)
y_pred = rf.predict(X_new)

In [ ]:
res = pd.DataFrame({'id':df_new['id'], 'booking_status':y_pred})
res.to_csv('/content/drive/MyDrive/12th Grade/Machine Learning/Data/Reservations/result2.csv', index=False)